In [3]:
#restart till get t4 or p4
!nvidia-smi

Sun Apr 18 09:58:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# #get 25 gb if necessary
# https://towardsdatascience.com/double-your-google-colab-ram-in-10-seconds-using-these-10-characters-efa636e646ff

In [5]:
# sources
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=G-kkz81OY6xH
#https://www.kaggle.com/samarthagarwal23/bert-lm-fine-tune-using-gpu-on-wikitext
#https://modelzoo.co/model/pytorch-pretrained-bert#Fine-tuning-with-BERT-running-the-examples
#https://towardsdatascience.com/how-to-use-bert-from-the-hugging-face-transformer-library-d373a22b0209
#https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=LTXXutqeDzPi


In [6]:
pip install transformers

     |████████████████████████████████| 2.1MB 9.0MB/s 
     |████████████████████████████████| 3.3MB 54.3MB/s 
     |████████████████████████████████| 870kB 57.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=d05838a7086cf5c98a048a65314822cd7291f759e65568adc29535c08c19dbe0
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [7]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

#a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc")

Some weights of the model checkpoint at SZTAKI-HLT/hubert-base-cc were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Transfer learning

In [9]:
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32001, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [11]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [12]:
model.num_parameters()
#110 millio!!

110651649

Prepare input

In [13]:
# !wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

In [14]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    # file_path="./oscar.eo.txt",
    file_path="./elokeszitett.txt",
    #file_path="./corpustext_openrefined_and_custom_v20.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 48.5 s, sys: 546 ms, total: 49 s
Wall time: 49.2 s


In [15]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Myfinetuded",
    overwrite_output_dir=True,
    num_train_epochs=12,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [23]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,0.834800
1000,0.757500
1500,0.735800
2000,0.720000
2500,0.705900
3000,0.703500
3500,0.694300
4000,0.684500
4500,0.683600
5000,0.679800


CPU times: user 4h 10min 17s, sys: 3h 6min 35s, total: 7h 16min 53s
Wall time: 3h 58min 36s


TrainOutput(global_step=20196, training_loss=0.6728782869136128, metrics={'train_runtime': 14316.2801, 'train_samples_per_second': 1.411, 'total_flos': 9.228836960973858e+16, 'epoch': 12.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 475136, 'train_mem_gpu_alloc_delta': 5017600, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 4547095040})

save the model

In [24]:
#save model
trainer.save_model("./Myfinetuded_mese")

check if mask does work

In [25]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./Myfinetuded_mese",
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
)

In [26]:
#test
fill_mask("a teherhordó lerakta " + tokenizer.mask_token +"erre a padra")

[{'score': 0.23142386972904205,
  'sequence': 'a teherhordó lerakta magát erre a padra',
  'token': 3768,
  'token_str': 'm a g á t'},
 {'score': 0.12321412563323975,
  'sequence': 'a teherhordó lerakta azt erre a padra',
  'token': 2333,
  'token_str': 'a z t'},
 {'score': 0.11774858087301254,
  'sequence': 'a teherhordó lerakta őket erre a padra',
  'token': 3478,
  'token_str': 'ő k e t'},
 {'score': 0.06864167749881744,
  'sequence': 'a teherhordó lerakta őt erre a padra',
  'token': 5315,
  'token_str': 'ő t'},
 {'score': 0.060181405395269394,
  'sequence': 'a teherhordó lerakta szemét erre a padra',
  'token': 10822,
  'token_str': 's z e m é t'}]

In [27]:
#test
fill_mask("ekkor a ház ura közelebb hívta magához és" + tokenizer.mask_token +"szóval beszélni") #nyájas

[{'score': 0.42594337463378906,
  'sequence': 'ekkor a ház ura közelebb hívta magához és néhány szóval beszélni',
  'token': 3370,
  'token_str': 'n é h á n y'},
 {'score': 0.2612355351448059,
  'sequence': 'ekkor a ház ura közelebb hívta magához és pár szóval beszélni',
  'token': 2983,
  'token_str': 'p á r'},
 {'score': 0.09124274551868439,
  'sequence': 'ekkor a ház ura közelebb hívta magához és szóval beszélni',
  'token': 0,
  'token_str': '[ P A D ]'},
 {'score': 0.026302428916096687,
  'sequence': 'ekkor a ház ura közelebb hívta magához és kedves szóval beszélni',
  'token': 5482,
  'token_str': 'k e d v e s'},
 {'score': 0.021895062178373337,
  'sequence': 'ekkor a ház ura közelebb hívta magához és szép szóval beszélni',
  'token': 3218,
  'token_str': 's z é p'}]

eval